In [1]:
#The following tutorial at https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk
#shows how to do sentiment analysis using the nltk toolkit for python. This was used for the comment dataset for NoSleep
#https://www.nltk.org/api/nltk.tokenize.html was also used
import pandas as pd
import re
import string
import random
import nltk
import numpy as np
import json

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.tokenize import TweetTokenizer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk import classify
from nltk import NaiveBayesClassifier

np.random.seed(2018)

In [ ]:
#This only has to be run once. It downloads the modules that are necessary
#to perform sentiment analysis

#The punkt module is a pre-trained model that helps you tokenize words and sentences.
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

In [2]:
#This loads the entire user Comments into a Panda DataFrame
#Note it has to be in the same directory as the notebook unless change
#This also fills all the blank sentiment scores to -1
comment_full = pd.read_csv('user_comments.csv')
comment_full['sentiment'] = comment_full['sentiment'].fillna(-1)
comment_full['prediction'] = -1

#comment_df = comment_full[:2001]
comment_df = comment_full
random_df = comment_full[5001:5101]

random_df = random_df.reset_index(drop=True)
random_df

,link_id,sortKey,score,permalink,author_fullname,id,storyId,author,parent_id,body,sentiment,prediction
0,t3_bbyzum,1555131600,3,/r/nosleep/comments/bbyzum/i_finally_got_my_re...,t2_154rlg,eknfnl9,bbyzum,Senpee,t1_ekndmde,r/wooooosh,-1,-1
1,t3_ddqdef,1570510800,1,/r/nosleep/comments/ddqdef/how_to_survive_camp...,t2_4e933umj,f2tug43,ddqdef,SlurpeeSlurper,t1_f2tsmpr,I'd keep away from him for a start. Good luck!,-1,-1
2,t3_enacmh,1578978000,1,/r/nosleep/comments/enacmh/my_mom_hired_a_clow...,t2_4x1l8o7v,fe2hmu8,enacmh,HesAPhantom304,t1_fe2hb55,Oh. Sorry then. I just got a little confused f...,-1,-1
3,t3_aby8yi,1546664400,1,/r/nosleep/comments/aby8yi/i_woke_up_to_find_a...,t2_2cguwyzj,ed4xavl,aby8yi,PepperidgeFarmMembas,t3_aby8yi,You never opened your mother’s present....what...,-1,-1
4,t3_c3kr9t,1561525200,2,/r/nosleep/comments/c3kr9t/i_kill_evil_that_th...,t2_xv4yuqt,erxg37c,c3kr9t,Vaughawa,t3_c3kr9t,Amazing,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
95,t3_azw37m,1552539600,1,/r/nosleep/comments/azw37m/a_cat_kept_on_makin...,t2_y42gk,eidcu8a,azw37m,MJGOO,t1_eiayjsx,88 miles an hour!!!,-1,-1
96,t3_e4kgrj,1575349200,22,/r/nosleep/comments/e4kgrj/if_the_time_stops_a...,t2_3z2en751,f9ec3fk,e4kgrj,scps53770,t1_f9ebuvh,What the actual fuck did I just read Lmao!,-1,-1
97,t3_cufufd,1566709200,1,/r/nosleep/comments/cufufd/i_have_the_perfect_...,t2_22igqeln,exw0qwv,cufufd,bothicc,t1_exv7rvj,Me! Me! I want one!,-1,-1
98,t3_cmykhy,1565326800,1,/r/nosleep/comments/cmykhy/the_previous_tenant...,t2_102ihk,ew69zua,cmykhy,eggplantsaredope,t3_cmykhy,Really weird that you couldnt bring the garden...,-1,-1


In [ ]:
#This section cleans up the code by removing false comments as well as comments that provide
#little value (less < characters). The exception is key words like more/moar etc which indicates
#the story was good.
#THIS ONLY NEEDS TO BE RUN ONCE AS IT SAVES THE INFO BACK IN THE ORIGINAL FILE

comment_df = comment_full
comment_df['body'] = comment_df['body'].str.strip()
comment_c = comment_df[~comment_df['body'].str.contains('remove')]
comment_c = comment_c[~(comment_c['body'].str.lower()).str.contains('cake day')]
comment_c = comment_c[~comment_c['body'].str.contains('It looks like there may be more to this story')]
comment_c = comment_c[~comment_c['body'].str.contains('https://red')]
comment_c = comment_c[comment_c['body'].str.len() >= 5]

comment_more = comment_df[(comment_df['body'].str.len() < 5) & ((comment_df['body'].str.lower()).str.contains('more'))]
comment_more["sentiment"] = 1
comment_moar = comment_df[(comment_df['body'].str.len() < 5) & ((comment_df['body'].str.lower()).str.contains('moar'))]
comment_moar["sentiment"] = 1
comment_sick = comment_df[(comment_df['body'].str.len() < 5) & ((comment_df['body'].str.lower()).str.contains('sick'))]
comment_sick["sentiment"] = 1
comment_holy = comment_df[(comment_df['body'].str.len() < 5) & ((comment_df['body'].str.lower()).str.contains('holy'))]
comment_holy["sentiment"] = 1

comment_c = pd.concat([comment_c, comment_more])
comment_c = pd.concat([comment_c, comment_moar])
comment_c = pd.concat([comment_c, comment_sick])
comment_c = pd.concat([comment_c, comment_holy])

comment_df = comment_c

comment_c.to_csv (r'user_comments.csv', index = False, header=True)

In [ ]:
#This takes the rated comments and splits them into a positive
#and negative data set. It then uses NLTK tweet tokenizer
#to tokenize the comments of each user storing them into
#a list of bag of words
#NB - Positive Comments = 1. 'Not' Positive Comments = 0
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

positive_comments = comment_df[comment_df['sentiment'] == 1]
negative_comments = comment_df[comment_df['sentiment'] == 0]

positive_tokens = []
negative_tokens = []

for count in range(len(positive_comments)):
    positive_tokens.append(tknzr.tokenize(positive_comments.iloc[count]["body"]))
    
for count in range(len(negative_comments)):
    negative_tokens.append(tknzr.tokenize(negative_comments.iloc[count]["body"]))

In [7]:
#This section cleanses and normalizes the positive and negative tokens obtained above
#Note that the cleansing part still needs some work.
def remove_noise(comment_tokens, stop_words = ()):
    cleaned_tokens = []

    for token, tag in pos_tag(comment_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token != '...' and token != '…' and token != '’' and token != '‘' and token != '..' and token not in string.punctuation and token.lower() not in stop_words and token.lower() != "#x200b":            
            cleaned_tokens.append(token.lower())
            
    return cleaned_tokens

In [ ]:
#This loads nltk's stopwords
stop_words = stopwords.words('english')
negative_tokens_c = []
positive_tokens_c = []

for count in range(len(negative_tokens)):
    negative_tokens_c.append(remove_noise(negative_tokens[count], stop_words))
    
for count in range(len(positive_tokens)):
    positive_tokens_c.append(remove_noise(positive_tokens[count], stop_words))

In [ ]:
#This section takes the cleansed tokens and put them into
#dictionary objects to be used later on in the models
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token
            
def get_comments_for_model(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tokens)

pos_tokens_mod = get_comments_for_model(positive_tokens_c)
neg_tokens_mod = get_comments_for_model(negative_tokens_c)

#Below shows the top 10 common words in the Positive and
#Negative Bag of words. This can be commented out once satisfied
all_pos_words = get_all_words(positive_tokens_c)
freq_dist_pos = FreqDist(all_pos_words)

all_neg_words = get_all_words(negative_tokens_c)
freq_dist_neg = FreqDist(all_neg_words)

print(freq_dist_pos.most_common(10))
print(freq_dist_neg.most_common(10))

In [ ]:
#NB - You CAN'T run this multiple times in a row. It basically removes the data from the
# *_tokens_mod variables. If you need to rerun this, please rerun the code above first
# to repopulate these variables

#This takes the positive/negative dictionaries created above and put them into data sets
#It then merges them together, shuffle them and create train/test versions of the data sets
#using 70/30% of the total data set respectively
positive_dataset = [(comment_dict, "Positive")
                     for comment_dict in pos_tokens_mod]

negative_dataset = [(comment_dict, "Negative")
                     for comment_dict in neg_tokens_mod]

dataset = positive_dataset + negative_dataset
train_size = int(len(dataset)*0.8)

random.shuffle(dataset)

train_data = dataset[:train_size]
test_data = dataset[train_size:]

#This is a sanity check to ensure that the total # equal to the comments that were rated
print(len(positive_dataset), len(negative_dataset), len(dataset))
print(len(train_data), len(test_data))

In [ ]:
#This stores the train and test data sets created into JSON files. These
#files can be used later on to load into the model without having to go through
#the work of cleaning up the data etc again
with open('model_train.json', 'w') as json_file:
    json.dump(train_data, json_file)

with open('model_test.json', 'w') as json_file:
    json.dump(test_data, json_file)

In [3]:
#This loads the train and test data from the json files created into the
#final objects to be loaded into the NaiveBayesClassifier model
with open('model_train.json') as json_file:
    train_data = json.load(json_file)

with open('model_test.json') as json_file:
    test_data = json.load(json_file)

In [5]:
#This creates a NaiveBayesClassifier model based on the train_data created
#above then it tests it using the test_data set. Ideally, we want the accuracy
#to be as high as possible. Right now it is about 55%
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

Accuracy is: 0.8046511627906977
Most Informative Features
                    holy = True           Positi : Negati =     16.4 : 1.0
                 amazing = True           Positi : Negati =     13.8 : 1.0
                   swear = True           Positi : Negati =      8.8 : 1.0
                   await = True           Positi : Negati =      8.8 : 1.0
                  writer = True           Positi : Negati =      8.8 : 1.0
               fantastic = True           Positi : Negati =      8.1 : 1.0
                   great = True           Positi : Negati =      8.1 : 1.0
                   could = True           Negati : Positi =      7.6 : 1.0
                 awesome = True           Positi : Negati =      7.0 : 1.0
                    hook = True           Positi : Negati =      6.8 : 1.0
None


In [8]:
#This section creates a new dataframe based on the un-rated comments
#and then apply the model on the 1st 10 comments. The results of the analysis
#is then shown with each comment. Once the model is more robust, this can
#be updated to run against the entire comment list with the predicted value
#stored in the prediction column
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

predictions = []

for count in range(100):
    custom_tokens = remove_noise(tknzr.tokenize(random_df.iloc[count]["body"]))
    predicted_sentiment = classifier.classify(dict([token, True] for token in custom_tokens))    
    
    predictions.append(1 if predicted_sentiment == 'Positive' else 0)

random_df['prediction'] = predictions
    
random_df.to_csv (r'user_comments_model_val.csv', index = False, header=True)

In [ ]:
#This section creates a new dataframe based on the un-rated comments
#and then apply the model on the 1st 10 comments. The results of the analysis
#is then shown with each comment. Once the model is more robust, this can
#be updated to run against the entire comment list with the predicted value
#stored in the prediction column
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

predictions = []
comment_df = comment_full

for count in range(len(comment_df)):
    if (comment_df.iloc[count]["sentiment"] == 1 or comment_df.iloc[count]["sentiment"] == 0):
        predictions.append(comment_df.iloc[count]["sentiment"])
        continue
        
    custom_tokens = remove_noise(tknzr.tokenize(comment_df.iloc[count]["body"]))
    predicted_sentiment = classifier.classify(dict([token, True] for token in custom_tokens))    
    
    predictions.append(1 if predicted_sentiment == 'Positive' else 0)

comment_df['prediction'] = predictions
    
comment_df.to_csv (r'user_comments.csv', index = False, header=True)